In [51]:
from IPython.display import Audio
import librosa
import IPython, numpy as np
from ly import *
import os

standard_sr = 44100
sr = 44100

In [52]:
def xml_to_ly(file, p = False):
    '''
    creates ly file to local dir
    
    INPUTS: 
    file: string to file name of XML in Testing_Data directory
    p: bool whether to print command line response
    
    
    '''

    
    #create lilypond file
    file = './Testing_Data/'+file
    cmd = 'musicxml2ly -a '+file 
    returned_value = os.system(cmd)
    if returned_value != 0:
        raise ValueError('File Not Found')
    if p ==  True:
        print('returned value:', returned_value)
    
    
    

In [53]:
def ly_to_text(filepath):
    '''
    INPUTS:    
        filepath: string to file path of .ly file
    -----------------------------------------
    RETURNS:
        data: string of encoded text from .ly file
    '''
    with open(filepath, encoding = 'cp1252') as f: #Might have to change encoding depending on how the mac version is
        data = f.read()
    f.close()
    return data

All nodes (instances of Item) have a ‘position’ attribute that indicates where the item starts in the source text. Almost
all items have the token that starts the expression in the ‘token’ attribute and possibly other tokens in the ‘tokens’
attribute, as a tuple.
The ‘end_position()’ method returns the position where the node (including its child nodes) ends.

In [54]:
def parse_PPOVO(bpm, PPOVO):
    '''
    INPUTS:
        bpm: beats per minute integer
        PPOVO: string of ly file containing note information
    --------------------------------
    RETURNS:
        onset_times: array of all float ground truth onset times
    '''
    time = 0.0
    onset_times = []
    prev = None
    i = 0
    is_tuple = False    
    tuple_fraction = 1
    #parse through PPOVO section
    while i < len(PPOVO)-2:
        #c -> current char in .ly text
        c = PPOVO[i]
        c_next = PPOVO[i+1]
        c_next_dig = c_next.isdigit()
        
        if(c == "\\"): #check for override
            i+=1
            tuplecheck = PPOVO.find("override TupletBracket",i) #check for tuple override
            if(tuplecheck == i): #tuple override found
                i = PPOVO.find("times",i)
                i += 6 #skip to find ft
                n_buffer = ""
                d_buffer = ""
                c = PPOVO[i]
                
                while c != "/": #get ft numerator
                    if(c.isdigit()):
                        n_buffer += c
                    i +=1
                    c = PPOVO[i]                  
                
                while c != " ": #get ft denominator
                    if(c.isdigit()):
                        d_buffer += c
                    i +=1
                    c = PPOVO[i]
                tuple_fraction = float(n_buffer)/float(d_buffer)
                is_tuple = True
                
        elif(is_tuple and c == "}"): #check for end of tuple overide, reset values
            is_tuple = False
            tuple_fraction = 1
            
        elif(c == "'" or (c == "r" and c_next_dig)): #note or rest found    
            #add note onset
            t = None
            if(c == "'"):
                t = "note"
                onset_times.append(time)               
                #parse through note length indicator
                while(c == "'"):
                    i += 1
                    c = PPOVO[i]
            else:
                t = "rest"
                i+=1
                c = PPOVO[i]
                
            #buffer to get note/rest value
            buffer = ""
            #keep going until total note len is found (1,2,4,8th,16th,32th note etc)
            while (c.isdigit()):
                buffer+=c
                i+=1
                c = PPOVO[i]
                
            if(c == "."): #dotted note
                tuple_fraction = 1.5
                
            #convert buffer to int to get note type
            if(buffer != ""):
                note_val = int(buffer)
                duration = note_to_seconds(bpm, note_val,tf=tuple_fraction)
                time+=duration
                
            if(c == "."): #dotted note
                tuple_fraction = 1.0
        i+=1
            
    return onset_times
            
                
        

        
    

In [55]:
def note_to_seconds(bpm, note_val,tf=1):
    '''
    INPUTS: 
        beats per minute integer
        note_val: type of note in float 
            1.0 = whole note
            0.5 = half note
            etc...
        tf: tuple fraction indicated in ly file
        
    -------------------------------
    RETURNS:
        duration: note duration in seconds
    '''
    duration = (60.0/bpm)*(4.0/note_val)*tf
    return duration

In [56]:
def ly_onsets(bpm, data):
    '''
    INPUTS:
        bpm: beats per minute integer
        data: string of ly file
    --------------------------------
    RETURNS:
        onset_times: array of floats for time stamps ground truth onsets
    '''
    #skip header information
    start = data.find('PartPOneVoiceOne') 
    #improper file type
    if start == -1:
        raise ValueError('Improper File Format. File is not Monophonic')
        
    #adjust start to begin at PPOVO node
    start +=  16
    while data[start] != '{':
        start += 1
        
    PPOVO = data[start:(len(data)-1)]
    
    #parse through to find note section
    onset_times = parse_PPOVO(bpm, PPOVO)
    
    return np.array(onset_times)
            
    

In [77]:
def extract_user_rhythm(signal):
    '''
    Inputs: audio signal (1D np.array)
    Outputs: times (sec) of onsets (1D np.array)

    Uses librosa's onset_detection to extract a 1D np.array
    of the onsets in an audio signal

    '''
    signal +=".wav"
    signal = "Testing_Data//" + signal
    signal , sr = librosa.core.load(signal)
    signal = librosa.util.normalize(signal, norm=2)
    rhythm_arr = librosa.onset.onset_detect(signal, sr=sr,units='time')
    
    return rhythm_arr


In [78]:
def search_onsets(onsets, lo, hi):
    """
    Binary search helper function to get user onset closest to target onset

    Inputs: array of onsets, lower leniency bound, upper leniency bound
    Outputs: array index of user onset closest to target onset, whether user onset is within leniency range
    """

    l = 0
    r = len(onsets) - 1
    while l <= r:

        m = int(l + (r - l) / 2)

        if lo <= onsets[m] <= hi:
            return m, True

        elif onsets[m] < hi:
            l = m + 1

        else:
            r = m - 1

    return m - 1, False


def compare_onsets(user_onsets, actual_onsets, leniency_len, bpm):
    '''
    Inputs: 
        user_onset: array of user onset in seconds
        actual_signal: array of user onsets in seconds
        leniency_len: note value for error window
        BPM: BPM of ground truth 
    Outputs: evaluation score based on number of correctly placed onsets,
             error of user onset in seconds,
             timesteps of user onset errors
    '''
    error_margins = []
    error_timestamps = []
    error_types = []
    score = None
    
    #no onsets case
    if len(user_onsets) == 0:
        score = 0
        error_timestamps = actual_onsets
        for i in range(0,len(actual_onsets)):
            error_types.append("Missing Note")
        return score, error_margins, error_timestamps , error_types
        
    
    #convert leniency_len to time
    l2 = leniency_len/2
    l1 = note_to_seconds(bpm, leniency_len)
    l2 = note_to_seconds(bpm,l2)
    
    # align first onset to time 0
    user_onsets = np.subtract(user_onsets, user_onsets[0])
    actual_onsets = np.subtract(actual_onsets, actual_onsets[0])
    print("adjusted user_onsets:",user_onsets)
    print("len of adjust user_onsets",len(user_onsets))
    print("adjusted actual_onsets", actual_onsets)
    print("len of adjust actual_onsets",len(actual_onsets))
    last_ind = 0
    for i in range(1, len(actual_onsets)):
        # aligned = user_onsets[(user_onsets >= actual_onsets[i]-leniency) & (user_onsets <= actual_onsets[i]+leniency)]
        # if len(aligned) > 0:
        # if at least than one user onset is detected to fall within the leniency region around the actual onset
        #    error_margins.append(0)

        closest_onset = search_onsets(user_onsets, actual_onsets[i]-l1, actual_onsets[i]+l1)
        closest_onset_1 = search_onsets(user_onsets, actual_onsets[i]-l2, actual_onsets[i]+l2)
        if closest_onset[1]: # if onset is within leniency range
#             error_margins.append(0)
            donothing = True
        else:
            ind = closest_onset[0]
            if ind != last_ind:
                error_timestamps.append(round(user_onsets[ind], 2))
                error_margins.append(round(user_onsets[ind] - actual_onsets[i], 2))
                if closest_onset_1[1]:
                    if ind<0:
                        error_types.append("Early")
                    else:
                        error_types.append("Late")
                else:
                    error_types.append("Extra Note")
            else:
                error_types.append("Missing Note")
                
                
            last_ind = ind

    score = 100 - (len(error_timestamps) / actual_onsets.size) * 100

    return score, error_margins, error_timestamps , error_types


^"text here to annotate as box" 

In [69]:
def ly_markup(error_margins, error_timestamps, error_types, filename, output_name, bpm):
    #open ly file
    ly_file = filename+".ly"
    data = ly_to_text(ly_file)
    #create new txt file for markup
    filename = output_name+"_markup.ly"
    markup = open(filename,"w+")
    
    #skip header information
    start = data.find('PartPOneVoiceOne') 
    
    #improper file type
    if start == -1:
        raise ValueError('Improper File Format. File is not Monophonic')
        
    #adjust start to begin at PPOVO node
    start +=  16
    while data[start] != '{':
        start += 1
    
   
    #PPOVO data section
    PPOVO = data[start:(len(data)-1)]
    header = data[:start]
    
    
    #PPOVO parsing information
    time = 0.0
    prev = None
    i = 0
    is_tuple = False    
    tuple_fraction = 1
    
    #error tracking information
    j = 0
    errors = True
    if len(error_types) == 0:
        errors = False
        
    #parse through PPOVO section if errors exist
    if errors:
        while i < len(PPOVO)-2:
            #c -> current char in .ly text
            c = PPOVO[i]
            c_next = PPOVO[i+1]
            c_next_dig = c_next.isdigit()

            if(c == "\\"): #check for override
                i+=1
                c = PPOVO[i]

                tuplecheck = PPOVO.find("override TupletBracket",i) #check for tuple override

                if(tuplecheck == i): #tuple override found
                    i = PPOVO.find("times",i)
                    i += 6 #skip to find ft
                    n_buffer = ""
                    d_buffer = ""
                    c = PPOVO[i]

                    while c != "/": #get ft numerator
                        if(c.isdigit()):
                            n_buffer += c
                        i +=1
                        c = PPOVO[i]                  

                    while c != " ": #get ft denominator
                        if(c.isdigit()):
                            d_buffer += c
                        i +=1
                        c = PPOVO[i]
                    c =  PPOVO[i]
                    tuple_fraction = float(n_buffer)/float(d_buffer)
                    is_tuple = True

            elif(is_tuple and c == "}"): #check for end of tuple overide, reset values
                is_tuple = False
                tuple_fraction = 1
                
            c = PPOVO[i]
            if(c == "'" or (c == "r" and c_next_dig)): #note or rest found
                
                #add note onset
                t = None
                if(c == "'"):
                    t = "note"             
                    #parse through note length indicator
                    while(c == "'"):
                        i += 1
                        c = PPOVO[i]
                else:
                    t = "rest"
                    i+=1
                    c = PPOVO[i]
                    
                #buffer to get note/rest value
                buffer = ""
                #keep going until total note len is found (1,2,4,8th,16th,32th note etc)
                while (c.isdigit()):
                    buffer+=c
                    i+=1
                    c = PPOVO[i]

                if(c == "."): #dotted note
                    tuple_fraction = 1.5

                #convert buffer to int to get note type
                
                if(buffer != ""):
                    note_val = int(buffer)
                    duration = note_to_seconds(bpm, note_val,tf=tuple_fraction)
                    
                    #error marking
                    error_time = float('inf')
                    if j < len(error_timestamps):
                        error_time = error_timestamps[j] 


                    while error_time <= time and j < len(error_timestamps):
#                         print("error time:",error_time)
#                         print("time: ",time)
                        error_message = " "+"^\"" + error_types[j] + "\"" + " "
                        PPOVO = PPOVO[:i] + error_message + PPOVO[i:]
                        i += len(error_message)
                        j+=1
                        if j < len(error_timestamps):
                            error_time = error_timestamps[j]
                    
                    
                    time+=duration
                        


                if(c == "."): #dotted note post correction
                    tuple_fraction = 1.0
                    
            i+=1
    data = header + PPOVO
    markup.write(data)
    markup.close()
    


In [70]:
def algoRhythm(file, user_signal, bpm, leniency):
    '''
    INPUTS:
        file: name of .xml file in Testing_Data directory
        user_signal: name of .wav file in Testing_Data directory of user performance
        BPM: beats per minute the rhythmic accuracy will be evaluated at
        leniency: integer 1-5 for how strict the accuracy guideline will be
        
    OUTPUTS:
        score: float between 0-100 of users accuracy
        
        
    '''
    xml_file = file+ ".xml"
    ly_file = file+".ly"
    
    #convert .xml file to .ly
    xml_to_ly(xml_file, p = False)
    
    #extract text from .ly file
    data = ly_to_text(ly_file)
    
    #extract truth onsets from .ly file
    true_onsets = ly_onsets(80, data)

    #extract user onsets from .wav file
    user_onsets = extract_user_rhythm(user_signal)
       
    #convert leniency value to note corresponding note duration
    l_note = None
    if(leniency < 1  or leniency > 5):
        raise ValueError("leniency must be between 1 and 5")
    else:
        if leniency == 1:
            l_note = 256
        elif leniency == 2:
            l_note = 128
        elif leniency == 3:
            l_note = 64
        elif leniency == 4:
            l_note = 32
        else:
            l_note = 16
    
    
    #compare rhythms of onsets and get errors
    score, error_margins, error_timestamps, error_types = compare_onsets(user_onsets,true_onsets,l_note,bpm)
    

    #markup .ly file with mistakes
    ly_markup(error_margins, error_timestamps, error_types, file, user_signal, bpm) 
    
    return score
    

In [80]:
s2 = algoRhythm("Ex1","Ex1_80bmp_midi_piano",80,5)


adjusted user_onsets: [ 0.          0.7662585   1.13777778  1.50929705  1.88081633  2.2523356
  2.5077551   2.7631746   3.0185941   3.39011338  3.76163265  4.5046712
  5.24770975  5.45668934  5.64244898  6.01396825  6.38548753  6.7570068
  7.12852608  7.50004535  8.26630385 10.12390023 10.4954195 ]
len of adjust user_onsets 23
adjusted actual_onsets [ 0.      0.75    1.5     1.875   2.625   3.      3.25    3.5     3.75
  4.125   4.5     5.25    6.      6.1875  6.375   6.75    7.125   7.875
  8.25    9.     10.875 ]
len of adjust actual_onsets 21
